In [43]:
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
import numpy as np

In [44]:
dataset = dgl.data.CoraGraphDataset()
print(f"Number of categories: {dataset.num_classes}")

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Number of categories: 7


In [45]:
g = dataset[0]
print("Node features")
print(g.ndata)
print("Edge features")
print(g.edata)

Node features
{'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'train_mask': tensor([ True,  True,  True,  ..., False, False, False])}
Edge features
{}


In [46]:
#torch.manual_seed(42)
#np.random.seed(42)
#model = torch_geometric.nn.GraphSAGE(g.ndata["feat"].shape[1], 16,dataset.num_classes,out_channels=7,jk='max')
edge_index_tensor = torch.stack(g.edges(), dim=0)

In [47]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    val_mask = g.ndata["val_mask"]
    test_mask = g.ndata["test_mask"]
    for e in range(100):
        # Forward
        logits = model(features,edge_index_tensor)

        # Compute prediction
        #pred = logits.argmax(1)
        probabilities=F.softmax(logits,dim=1)
        #print(probabilities.shape)
        pred=probabilities.argmax(1)
        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print(
                f"In epoch {e}, loss: {loss:.3f}, val acc: {val_acc:.3f} (best {best_val_acc:.3f}), test acc: {test_acc:.3f} (best {best_test_acc:.3f})"
            )

In [54]:
np.random.seed(42)

# Set seed for PyTorch
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

np.random.seed(42)
model = torch_geometric.nn.GraphSAGE(g.ndata["feat"].shape[1], 16, dataset.num_classes,jk='cat')
train(g, model)

In epoch 0, loss: 2.756, val acc: 0.122 (best 0.122), test acc: 0.130 (best 0.130)
In epoch 5, loss: 2.247, val acc: 0.202 (best 0.202), test acc: 0.194 (best 0.194)
In epoch 10, loss: 1.847, val acc: 0.200 (best 0.202), test acc: 0.198 (best 0.194)
In epoch 15, loss: 1.653, val acc: 0.170 (best 0.274), test acc: 0.201 (best 0.260)
In epoch 20, loss: 1.227, val acc: 0.612 (best 0.612), test acc: 0.593 (best 0.593)
In epoch 25, loss: 0.577, val acc: 0.602 (best 0.628), test acc: 0.598 (best 0.595)
In epoch 30, loss: 0.166, val acc: 0.624 (best 0.656), test acc: 0.584 (best 0.618)
In epoch 35, loss: 0.027, val acc: 0.608 (best 0.656), test acc: 0.609 (best 0.618)
In epoch 40, loss: 0.005, val acc: 0.618 (best 0.656), test acc: 0.607 (best 0.618)
In epoch 45, loss: 0.001, val acc: 0.604 (best 0.656), test acc: 0.606 (best 0.618)
In epoch 50, loss: 0.000, val acc: 0.608 (best 0.656), test acc: 0.599 (best 0.618)
In epoch 55, loss: 0.000, val acc: 0.622 (best 0.656), test acc: 0.626 (best 0

In [ ]:
import numpy as np
torch.manual_seed(42)
np.random.seed(42)
model = torch_geometric.nn.GraphSAGE(g.ndata["feat"].shape[1], 16, dataset.num_classes,jk=None,out_channels=7)
edge_index_tensor = torch.stack(g.edges(), dim=0)
train(g, model)